In [ ]:
!git clone https://github.com/seanwoory/MALTbot

In [ ]:
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

In [ ]:
%%bash

set -euo pipefail
REPO_DIR="/content/MALTbot"
REPO_URL="https://github.com/seanwoory/MALTbot.git"

if [ -d "${REPO_DIR}/.git" ]; then
  echo "[info] Repo exists. Pull latest main..."
  cd "${REPO_DIR}"
  git fetch origin
  git checkout main || true
  git pull --ff-only origin main
else
  echo "[info] Cloning repo..."
  git clone "${REPO_URL}" "${REPO_DIR}"
  cd "${REPO_DIR}"
fi

pwd
git rev-parse --short HEAD

In [ ]:
%%bash
set -euo pipefail
python -m pip install -U pip setuptools wheel

# 1) 먼저 최신 호환 스택 설치 (Py3.12 compatible)
python -m pip install \
  "numpy" \
  "scipy>=1.13.0" \
  "pandas" \
  "scikit-learn>=1.5" \
  "pymatgen" \
  "matminer==0.7.4" \
  "torch==2.5.1" \
  "tqdm" "pyyaml"

# 2) matbench는 의존성 설치를 스킵하고 본체만 설치
python -m pip install "matbench==0.6" --no-deps

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
%%bash

set -euo pipefail
cd /content/MALTbot
python scripts/run_chgnet_mp_e_form.py --config configs/chgnet_mp_e_form.yaml

In [ ]:
%%bash

set -euo pipefail
cd /content/MALTbot
find results -type f -name "results.json" | sort

In [ ]:
from google.colab import userdata
import os

token = userdata.get('GH_TOKEN')
if not token:
    raise ValueError("Colab Secret 'GH_TOKEN'이 비어 있습니다.")
os.environ['GH_TOKEN'] = token
print("GH_TOKEN loaded from Colab Secrets")

In [ ]:
!echo "GH_TOKEN length: ${#GH_TOKEN}"

In [ ]:
%%bash
set -euo pipefail
cd /content/MALTbot

: "${GH_TOKEN:?GH_TOKEN is not set}"   # 토큰 없으면 여기서 즉시 에러로 종료

git config user.name "colab-bot"
git config user.email "colab-bot@users.noreply.github.com"

DATE_KST="$(TZ=Asia/Seoul date +%Y%m%d)"
RUN_TAG="routeb-chgnet-mp-e-form"
BRANCH="bot/${DATE_KST}-${RUN_TAG}"

git checkout -B "${BRANCH}"

[ -f RESULTS.md ] && git add RESULTS.md || true
git add results/daily/** || true

if git diff --cached --quiet; then
  echo "[info] Commit할 변경사항이 없습니다."
  exit 0
fi

git commit -m "results: ${RUN_TAG} ${DATE_KST}"

# Basic auth header (x-access-token)
AUTH="$(printf "x-access-token:%s" "$GH_TOKEN" | base64 -w0)"
git -c http.https://github.com/.extraheader="Authorization: Basic ${AUTH}" \
  push -u origin "${BRANCH}"

echo "[done] Pushed branch: ${BRANCH}"
echo "PR 생성: https://github.com/seanwoory/MALTbot/compare/main...${BRANCH}?expand=1"